In [1]:
import os
import numpy as np

## 5.19 Rotate a 2D Array 

Image rotation is a fundamental operation in computer graphics. Figure 5.4 illustrates the rotation operation on a 2D array representing a bit-map of an image. Specifically, the image is rotated by 90 degrees clockwise.

Write a function that takes as input an n x n 2D array, and rotates the array by 90 degrees clockwise.
Hint: Focus on the boundary elements.

In [102]:
def rotate_90_clock_np(arr):
    arr = np.array(arr)
    arr_rot = np.zeros(arr.shape)
    for old_row_ind, new_col_ind in enumerate(range(len(arr))[::-1]):
        arr_rot[:, new_col_ind] = arr[old_row_ind, :]
    return arr_rot

def rotate_90_clock(arr):
    n = len(arr)
    arr_rot = [[None] * n for i in range(n)]
    for old_row_ind, new_col_ind in enumerate(range(n)[::-1]):
        for old_col_ind, new_row_ind in enumerate(range(n)):
            arr_rot[new_row_ind][new_col_ind] = arr[old_row_ind][old_col_ind]
    return arr_rot

In [103]:
# arr = [[i] * n for i in range(5)]
# arr = [[1, 2], [3, 4]]
arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]

arr_rot = rotate_90_clock(arr)

print(np.array(arr))
print('')
print(np.array(arr_rot))

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]

[[13  9  5  1]
 [14 10  6  2]
 [15 11  7  3]
 [16 12  8  4]]


## 5.13 Sample Online Data 

This problem is motivated by the disgn of a packet sniffer that provides a uniform sample of packets for a network session.

Design a program that takes as input a size k, and reads packets, coninuously maintaining a uniform random subset of size k of the read packets.

Hint: Suppose you have a procedure which selects k packets from the first n >= k packets as specified. How would you deal with the (n+1)st packet?

In [2]:
import random

class PacketSampler:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = float(self.k) / (self.n + 1)
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


class PacketSamplerWrong:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = random.random()
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


def get_packet_subset(n, packet_sampler):
    for i in range(n):
        packet_sampler.receive_packet(i)
    return packet_sampler.subset


def get_simulated_distribution(n, k, sampler_name, num_trials):
    if sampler_name == 'right':
        Sampler = PacketSampler
    elif sampler_name == 'wrong':
        Sampler = PacketSamplerWrong
    else:
        raise ValueError('Unrecognized sampler name.')
    
    packet_counter = {p: 0 for p in range(n)}
    for trial_ind in range(num_trials):
        subset = get_packet_subset(n, Sampler(k))
        for p in subset:
            packet_counter[p] += 1
    
    return packet_counter

In [5]:
num_trials = 100000
n = 10
k = 3
sampler_name = 'right'

packet_counts = get_simulated_distribution(n=n, k=k, sampler_name=sampler_name, num_trials=num_trials)
num_exp = num_trials / n * k

for count in packet_counts.itervalues():
    pct_dev = 100.0 * abs(num_exp - count) / num_exp
    print(pct_dev)

0.543333333333
0.00333333333333
0.0166666666667
0.233333333333
0.523333333333
1.10666666667
0.82
0.693333333333
0.71
0.0766666666667


In [19]:
num_exp

3000

In [105]:
k = 3
num_packet = 10

num_trial = 10000
packet_counter = {i: 0 for i in range(num_packet)}

for trial_ind in range(num_trial):

    ps = PacketSampler(k)

    for p in range(num_packet):
        ps.receive_packet(p)
        
    for p in ps.subset:
        packet_counter[p] += 1
        
packet_counter

{0: 3084,
 1: 2973,
 2: 2971,
 3: 3005,
 4: 3016,
 5: 2936,
 6: 2956,
 7: 3008,
 8: 2979,
 9: 3072}